In [1]:
import sqlite3

import brotli
import pandas as pd
from pyzstd import decompress
import json
from uuid import uuid4

Global Variables

In [2]:
# Global variables
BUILD_DATABASE = True
TRAIN = True

Helper Functions

In [3]:
def decode_zstd (value):
  value = decompress(value)
  value = json.loads(value)
  return value

def decode_brotli(data):
    data = brotli.decompress(data)
    return data


def connect_db(db_name):
    try:
        conn = sqlite3.connect(db_name)
        print("Connection to SQLite DB successful")
        return conn
    except sqlite3.Error as e:
        print("Connection to SQLite DB failed:", e)
    

def run_query(conn, query):
    try:
        df = pd.read_sql_query(query, conn)
        return df
    except sqlite3.Error as e:
        print("Query failed:", e)

Main

In [4]:
def create_dataset():
    data_sources = {
        # 'simon-duels': 'datasets\simon-duels.sqlite',
        'simon-ts': 'datasets\simon-two-snake.sqlite',
        }
    
    for source in data_sources:
        conn = connect_db(data_sources[source])
        conv_dataset = translate_dataset(conn, source)
        
        final_dataset = pd.concat([conv_dataset])
        
    return final_dataset
        
        
def main():
    if BUILD_DATABASE:
        dataset = create_dataset()
        # print(dataset)
    

Database translations

In [5]:

def translate_dataset(conn, source=False):
    if not source:
        print("Please delare database source")
        return False
    
    if source == "simon-ts":
        conv_dataset = translate_simon_ts(conn)
    
    conv_dataset = convert_game(conv_dataset)
    return conv_dataset

        
def translate_simon_ts(conn):
    query = "SELECT * FROM snake_games LIMIT 5"
    # Retrieve games dataset
    games_table = run_query(conn, query)
    # Decode game compression from brotli to json
    games_table['compressed_frames_json'] = games_table['compressed_frames_json'].apply(decode_brotli)
    # Rename columns to match final dataset
    games_table.rename(columns={
        "id": "id",
        "game_id": "game_id",
        "game_type": "game_mode",
        "unique_snake_count": "snake_count",
        'compressed_frames_json': 'record'
        }, inplace=True)
    
    return games_table
    
        
def convert_game(dataset):
    # Convert game to dataframe
    games = list(dataset['record'])
    
    for game_index in range(len(games)):
        game = json.loads(games[game_index])
        
        for frame_index in range(len(game)):
            frame = game[frame_index]
            
            
            
            print(frame)
            
            
    final_games = 0
    return final_games

In [7]:

conn = connect_db('datasets\simon-two-snake.sqlite')
conv_dataset = translate_dataset(conn, 'simon-ts')

Connection to SQLite DB successful
{'snakes': [{'body': [{'X': 6, 'Y': 9}, {'X': 5, 'Y': 9}, {'X': 4, 'Y': 9}, {'X': 3, 'Y': 9}, {'X': 3, 'Y': 8}, {'X': 4, 'Y': 8}, {'X': 5, 'Y': 8}], 'id': 'gs_px6r3B4fYhRbKBQ8Gx7HYVgJ', 'name': 'nomblegomble'}, {'body': [{'X': -1, 'Y': 6}, {'X': 0, 'Y': 6}, {'X': 1, 'Y': 6}, {'X': 2, 'Y': 6}, {'X': 3, 'Y': 6}, {'X': 4, 'Y': 6}, {'X': 5, 'Y': 6}], 'id': 'gs_c9SCqK6fqqRrcPp6T9gCwT96', 'name': 'Prüzze v2'}], 'food': [{'X': 3, 'Y': 10}, {'X': 10, 'Y': 0}, {'X': 9, 'Y': 7}, {'X': 7, 'Y': 10}], 'hazards': []}
{'snakes': [{'body': [{'X': 5, 'Y': 9}, {'X': 4, 'Y': 9}, {'X': 3, 'Y': 9}, {'X': 3, 'Y': 8}, {'X': 4, 'Y': 8}, {'X': 5, 'Y': 8}, {'X': 6, 'Y': 8}], 'id': 'gs_px6r3B4fYhRbKBQ8Gx7HYVgJ', 'name': 'nomblegomble'}, {'body': [{'X': 0, 'Y': 6}, {'X': 1, 'Y': 6}, {'X': 2, 'Y': 6}, {'X': 3, 'Y': 6}, {'X': 4, 'Y': 6}, {'X': 5, 'Y': 6}, {'X': 6, 'Y': 6}], 'id': 'gs_c9SCqK6fqqRrcPp6T9gCwT96', 'name': 'Prüzze v2'}], 'food': [{'X': 3, 'Y': 10}, {'X': 10, 'Y': 0}, {

In [8]:
{'snakes': [{'body': [{'X': 6, 'Y': 9}, {'X': 5, 'Y': 9}, {'X': 4, 'Y': 9}, {'X': 3, 'Y': 9}, {'X': 3, 'Y': 8}, {'X': 4, 'Y': 8}, {'X': 5, 'Y': 8}], 'id': 'gs_px6r3B4fYhRbKBQ8Gx7HYVgJ', 'name': 'nomblegomble'}, {'body': [{'X': -1, 'Y': 6}, {'X': 0, 'Y': 6}, {'X': 1, 'Y': 6}, {'X': 2, 'Y': 6}, {'X': 3, 'Y': 6}, {'X': 4, 'Y': 6}, {'X': 5, 'Y': 6}], 'id': 'gs_c9SCqK6fqqRrcPp6T9gCwT96', 'name': 'Prüzze v2'}], 'food': [{'X': 3, 'Y': 10}, {'X': 10, 'Y': 0}, {'X': 9, 'Y': 7}, {'X': 7, 'Y': 10}], 'hazards': []}

{'snakes': [{'body': [{'X': 6, 'Y': 9},
    {'X': 5, 'Y': 9},
    {'X': 4, 'Y': 9},
    {'X': 3, 'Y': 9},
    {'X': 3, 'Y': 8},
    {'X': 4, 'Y': 8},
    {'X': 5, 'Y': 8}],
   'id': 'gs_px6r3B4fYhRbKBQ8Gx7HYVgJ',
   'name': 'nomblegomble'},
  {'body': [{'X': -1, 'Y': 6},
    {'X': 0, 'Y': 6},
    {'X': 1, 'Y': 6},
    {'X': 2, 'Y': 6},
    {'X': 3, 'Y': 6},
    {'X': 4, 'Y': 6},
    {'X': 5, 'Y': 6}],
   'id': 'gs_c9SCqK6fqqRrcPp6T9gCwT96',
   'name': 'Prüzze v2'}],
 'food': [{'X': 3, 'Y': 10},
  {'X': 10, 'Y': 0},
  {'X': 9, 'Y': 7},
  {'X': 7, 'Y': 10}],
 'hazards': []}